In [1]:
import scipy as sp
import matplotlib.pyplot as plt

from kneed import KneeLocator
import math

import random

from scot import*

# For optimal transport operations:
import ot

# For pre-processing, normalization
from sklearn.preprocessing import StandardScaler, normalize

import numpy as np
import random, math, os, sys
from sklearn.metrics import roc_auc_score, silhouette_samples
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA


import warnings
warnings.filterwarnings("ignore")

import torch
from torch import nn, optim
from torch.autograd import grad
import torch.nn.functional as F




# For computing graph distances:
from sklearn.neighbors import NearestNeighbors,DistanceMetric, KNeighborsClassifier, kneighbors_graph


def model(X,y, cellTypes_X,cellTypes_y, labels, epsilon =1e-3):

    def kmeans_finder(X):
        
        # calculate the WSS for different number of clusters
        wss = []
        for k in range(1, 20):
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(X)
            wss.append(kmeans.inertia_)

        # find the elbow point using KneeLocator
        kl = KneeLocator(range(1, 20), wss, curve='convex', direction='decreasing')
        best_k = kl.elbow

#         # plot the WSS against the number of clusters with the elbow point
#         plt.plot(range(1, 20), wss)
#         plt.xlabel('Number of clusters')
#         plt.ylabel('Within-cluster sum of squares')
#         plt.title('Elbow method for optimal number of clusters')
#         plt.vlines(best_k, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
#         plt.show()

#         print('Best number of clusters:', best_k)


    
        # fit the KMeans model with k clusters
        #kmeans = KMeans(n_clusters= math.floor(X.shape[0] / best_k), random_state=42).fit(X)
        
        # best_k = 3
        
        kmeans = KMeans(n_clusters= best_k, random_state=42).fit(X)
    
        # get the centroids and labels
        centroids = kmeans.cluster_centers_
        labels = kmeans.labels_
    
        # loop through each centroid and find the index of the closest point
        closest_idx = []
        for i in range(len(centroids)):
            distances = np.linalg.norm(X - centroids[i], axis=1)
            closest_idx.append(np.argmin(distances))

        
#         # create a scatter plot of the data points
#         plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')
#         # add the cluster centroids as black crosses
#         plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', s=200, linewidths=3, color='black')
#         # add the closest index points as red dots
#         plt.scatter(X[closest_idx, 0], X[closest_idx, 1], marker='o', s=100, color='red')

#         plt.show()

        return best_k, closest_idx

    
    def calc_frac_idx(x1_mat,x2_mat):
        
        """
        Returns fraction closer than true match for each sample (as an array)
        """
        
        fracs = []
        x = []
        nsamp = x1_mat.shape[0]
        rank=0
        for row_idx in range(nsamp):
            euc_dist = np.sqrt(np.sum(np.square(np.subtract(x1_mat[row_idx,:], x2_mat)), axis=1))
            true_nbr = euc_dist[row_idx]
            sort_euc_dist = sorted(euc_dist)
            rank =sort_euc_dist.index(true_nbr)
            frac = float(rank)/(nsamp -1)

            fracs.append(frac)
            x.append(row_idx+1)

        return fracs,x

    def calc_domainAveraged_FOSCTTM(x1_mat, x2_mat):
        
        """
        Outputs average FOSCTTM measure (averaged over both domains)
        Get the fraction matched for all data points in both directions
        Averages the fractions in both directions for each data point
        """
    
        fracs1,xs = calc_frac_idx(x1_mat, x2_mat)
        fracs2,xs = calc_frac_idx(x2_mat, x1_mat)
        fracs = []
        for i in range(len(fracs1)):
            fracs.append((fracs1[i]+fracs2[i])/2)  
        return fracs

    def calc_sil(x1_mat,x2_mat,x1_lab,x2_lab):
        
        """
        Returns silhouette score for datasets with cell clusters
        """
    
        sil = []
        sil_d0 = []
        sil_d3 = []
        sil_d7 = []
        sil_d11 = []
        sil_npc = []

        x = np.concatenate((x1_mat,x2_mat))
        lab = np.concatenate((x1_lab,x2_lab))

        sil_score = silhouette_samples(x,lab)

        nsamp = x.shape[0]
        for i in range(nsamp):
            if(lab[i]==1):
                sil_d0.append(sil_score[i])
            elif(lab[i]==2):
                sil_d3.append(sil_score[i])
            elif(lab[i]==3):
                sil_d7.append(sil_score[i])
            elif(lab[i]==4):
                sil_d11.append(sil_score[i])
            elif(lab[i]==5):
                sil_npc.append(sil_score[i])

        avg = np.mean(sil_score)
        d0 = sum(sil_d0)/len(sil_d0)
        d3 = sum(sil_d3)/len(sil_d3)
        d7 = sum(sil_d7)/len(sil_d7)
        d11 = sum(sil_d11)/len(sil_d11)
        npc = sum(sil_npc)/len(sil_npc)

        return avg,d0,d3,d7,d11,npc

    def binarize_labels(label,x):
        
        """
        Helper function for calc_auc
        """
    
        bin_lab = np.array([1] * len(x))
        idx = np.where(x == label)

        bin_lab[idx] = 0
        return bin_lab


    def calc_auc(x1_mat, x2_mat, x1_lab, x2_lab):
        
        """
        calculate avg. ROC AUC scores for transformed data when there are >=2 number of clusters.
        """

        nsamp = x1_mat.shape[0]

        auc = []
        auc_d0 = []
        auc_d3 = []
        auc_d7 = []
        auc_d11 = []
        auc_npc = []

        for row_idx in range(nsamp):
   
            euc_dist = np.sqrt(np.sum(np.square(np.subtract(x1_mat[row_idx,:], x2_mat)), axis=1))
            y_scores = euc_dist
            y_true = binarize_labels(x1_lab[row_idx],x2_lab)

            auc_score = roc_auc_score(y_true, y_scores)
            auc.append(auc_score)
        
            if(x1_lab[row_idx]==0):
                auc_d0.append(auc_score)
            elif(x1_lab[row_idx]==1):
                auc_d3.append(auc_score)
            elif(x1_lab[row_idx]==2):
                auc_d7.append(auc_score)
            elif(x1_lab[row_idx]==3):
                auc_d11.append(auc_score)
            elif(x1_lab[row_idx]==4):
                auc_npc.append(auc_score)

        avg = sum(auc)/len(auc)
        d0 = sum(auc_d0)/len(auc_d0)
        d3 = sum(auc_d3)/len(auc_d3)
        d7 = sum(auc_d7)/len(auc_d7)
        d11 = sum(auc_d11)/len(auc_d11)
        npc = sum(auc_npc)/len(auc_npc)

        return avg,d0,d3,d7,d11,npc

    def transfer_accuracy(domain1, domain2, type1, type2, n):
    
        """
        Metric from UnionCom: "Label Transfer Accuracy"
        """
    
        knn = KNeighborsClassifier(n_neighbors=n)
        knn.fit(domain2, type2)
        type1_predict = knn.predict(domain1)
        np.savetxt("type1_predict.txt", type1_predict)
        count = 0
        for label1, label2 in zip(type1_predict, type1):
            if label1 == label2:
                count += 1
        return count / len(type1)


    def distance_matrix(X,y):
        
        dist = DistanceMetric.get_metric('euclidean')
        C1 = dist.pairwise(X)
        C2 = dist.pairwise(y)
        return C1, C2


    def finding_triplets(dist_matrix):
        
        _, closest_idx = kmeans_finder(dist_matrix)
      
    
        triplets = []
        
        # neigh = NearestNeighbors(n_neighbors=random.randint(2, dist_matrix.shape[0]))
        neigh = NearestNeighbors(n_neighbors=4)
        
        neigh.fit(dist_matrix)
        for i in range(len(closest_idx)):
            neigh_dist, neigh_ind = neigh.kneighbors([dist_matrix[closest_idx[i]]])
            j = neigh_ind[0][0]
            k = neigh_ind[0][1]
            l = neigh_ind[0][-1]
            triplets.append((j, k, l))
        return triplets



    def min_gromov_wasserstein_distance1(C1_fixed, C2_fixed, C1, C2, p, q, nb_iter_max=5, lr=1, epsilon = epsilon):
    
        loss_iter = []
        
        triplets_1 = finding_triplets(C1_fixed.detach().cpu().numpy())
        
        triplets_2 = finding_triplets(C2_fixed.detach().cpu().numpy())
       
        with torch.autograd.profiler.profile() as prof:
            for i in range(nb_iter_max):
                print('iter', i)

                loss1 = ot.gromov.entropic_gromov_wasserstein2(C1, C2, p, q, loss_fun="square_loss", epsilon = epsilon, tol=1e-02, verbose=True, log=False)
                
        
                loss4 = ot.gromov.entropic_gromov_wasserstein2(C1_fixed, C1, p, q, loss_fun="square_loss", epsilon = epsilon, tol=1e-02, verbose=True, log=False)

                loss5 = ot.gromov.entropic_gromov_wasserstein2(C2_fixed, C2, p, q, loss_fun="square_loss", epsilon = epsilon, tol=1e-02, verbose=True, log=False)
        
        
                total_loss1 = 0
                total_loss2 = 0
        
        
            # define the triplet margin loss function
            
                triplet_loss_fn = nn.TripletMarginLoss(margin=1)
            
                for triplet in triplets_1:
                
                    anchor1 = C1[triplet[0]]
                    pos1 = C1[triplet[1]] 
                    neg1 = C1[triplet[2]]
            
                    lossC1 = triplet_loss_fn(anchor1, pos1, neg1)
        
                    total_loss1 += lossC1
            
                triplet_loss_fn = nn.TripletMarginLoss(margin=1)
            
                for triplet in triplets_2:
                
                    anchor2 = C2[triplet[0]] 
                    pos2 = C2[triplet[1]] 
                    neg2 = C2[triplet[2]]
        
            
                    lossC2 = triplet_loss_fn(anchor2, pos2, neg2)
            
                    total_loss2 += lossC2
            
            
                loss = (loss1 + loss4+ loss5) + 1/len(triplets_1)*total_loss1 + 1/len(triplets_2)*total_loss2
                #print(loss)
                loss_iter.append(loss.clone().detach().cpu().numpy())
    

        # Compute the gradient of the loss with respect to the cost matrices
                
                loss.backward()
                with torch.no_grad():
                
                    grad_C1 = C1.grad
                
                    grad_C2 = C2.grad
    
     
                    C1 -= lr * grad_C1
                    C2 -= lr * grad_C2
            
            
                    for m in range(C1.shape[0]):
                        C1[m][m] = 0
             
                    for m in range(C2.shape[0]):
                        C2[m][m] = 0


                    C1.grad.zero_()
            
                    C2.grad.zero_()
            
        
        
        
        # Convert the final tensors back to numpy arrays
        C1 = C1.detach().cpu().numpy()
        C2 = C2.detach().cpu().numpy()
    
        return C1, C2, loss_iter
    
    X, y  = normalize(X, norm="l2"), normalize(y, norm="l2")
  
    C1, C2 = distance_matrix(X, y)
    
    
    C1_torch = torch.tensor(C1, dtype=torch.float32, requires_grad=True)
    C2_torch = torch.tensor(C2, dtype=torch.float32, requires_grad=True)
  
   
    
    p = ot.unif(C1.shape[0])
    p = torch.tensor(p, dtype=torch.float32)
    q = ot.unif(C2.shape[0])
    q = torch.tensor(q, dtype=torch.float32)
    
    
    C1_opt, C2_opt, loss_iter = min_gromov_wasserstein_distance1(C1_torch, C2_torch, C1_torch, C2_torch, p, q, nb_iter_max=10, lr=10,  epsilon = epsilon)
    

    plt.figure()
    plt.plot(loss_iter)
    plt.title("Loss along iterations")
    

    p = ot.unif(C1.shape[0])
    q = ot.unif(C2.shape[0])
    distance_C1_opt_C2_opt = ot.gromov.entropic_gromov_wasserstein2(C1_opt, C2_opt, p, q, loss_fun = 'square_loss', epsilon=1e-2)
    print('distance between C1_opt and C2_opt:', distance_C1_opt_C2_opt)

    
    p = ot.unif(C1.shape[0])
    q =  ot.unif(C2.shape[0])
    P = ot.gromov.entropic_gromov_wasserstein(C1_opt, C2_opt, p, q, loss_fun='square_loss', epsilon = epsilon, log=False, verbose=True)
 
    # projection
    #Projecting the first domain onto the second domain
    y_new=y
    weights=np.sum(P,axis = 0)
    X_new =np.matmul(P, y) / weights[:, None]
  

    # We will use the average FOSCTTM measure implemented in evals.py for evaluation (metric used in the publication Demetci et al 2021)
    # This measure reports the fraction of samples closer to a sample than its true match (FOSCTTM), averaged over all samples. 
    fracs= calc_domainAveraged_FOSCTTM(X_new, y_new)
    print("Average FOSCTTM score for this alignment with X onto Y is: ", np.mean(fracs))
    

    
    #Plotting sorted FOSCTTM to show the distributions of FOSCTTM across cells:
    plt.figure()
    legend_label="SCOT alignment FOSCTTM \n average value: "+str(np.mean(fracs)) #Put average FOSCTTM in the legend
    plt.plot(np.arange(len(fracs)), np.sort(fracs), "r--", label=legend_label)
    plt.legend()
    plt.xlabel("Cells")
    plt.ylabel("Sorted FOSCTTM")
    plt.show()

    
    
    
    #Reduce the dimensionality of the aligned domains to two (2D) via PCA for the sake of visualization:
    pca=PCA(n_components=2)
    Xy_pca=pca.fit_transform(np.concatenate((X_new, y_new), axis=0))
    X_pca=Xy_pca[0: X.shape[0],]
    y_pca=Xy_pca[X.shape[0]:,]

    #Plot aligned domains, samples colored by domain identity:
    plt.figure()
    plt.scatter(X_pca[:,0], X_pca[:,1], c="k", s=15, label="Chromatin Accessibility")
    plt.scatter(y_pca[:,0], y_pca[:,1], c="r", s=15, label="Gene Expression")
    plt.legend()
    plt.title("Colored based on domains")
    plt.show()

    
    
    
    colormap = plt.get_cmap('rainbow', len(labels)) 
    plt.scatter(X_pca[:,0], X_pca[:,1], c=cellTypes_X, s=15, cmap=colormap)
    plt.scatter(y_pca[:,0], y_pca[:,1], c=cellTypes_y, s=15, cmap=colormap)
    # plt.colorbar()
    cbar=plt.colorbar()

    
    # approximately center the colors on the colorbar when adding cell type labels
    fraction_for_colorbar = (len(labels)-1)/(len(labels))
    tick_locs = (np.arange(1,len(labels)+1)+fraction_for_colorbar) * fraction_for_colorbar 
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels(labels) #cell-type labels
    plt.title("Colored based on cell type identity")
    plt.show()




    originalX_pca=pca.fit_transform(X)
    originaly_pca=pca.fit_transform(y)

    #Visualization of the global geometry
    fig, (ax1, ax2)= plt.subplots(1,2)
    ax1.scatter(originalX_pca[:,0], originalX_pca[:,1], c="k", s=15)
    ax1.set_title("Chromatin Accessibiliy Domain \n *before* Alignment")
    ax2.scatter(originaly_pca[:,0], originaly_pca[:,1], c="r", s=15)
    ax2.set_title("Gene Expression Domain \n *before* Alignment")
    plt.show()

    
    #Visualization of the cell type clusters in original domains *before* alignment
    fig, (ax1, ax2)= plt.subplots(1,2)

    fig1= ax1.scatter(originalX_pca[:,0], originalX_pca[:,1], c=cellTypes_X, s=15, cmap=colormap)
    ax1.set_title("Chromatin Accessibiliy \n *before* Alignment")

    fig2= ax2.scatter(originaly_pca[:,0], originaly_pca[:,1],  c=cellTypes_y, s=15, cmap=colormap)
    ax2.set_title("Gene Expression Domain \n *before* Alignment")

    cbar=fig.colorbar(fig2)
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels(labels) #cell-type labels



    return C1_opt, C2_opt

In [2]:
def data_generator():
    
    # scrna=np.load("../../data/scrna_feat.npy")
    scrna=np.load("../data/scrna_feat.npy")
    
    # Define the logarithmic transformation function
    non_linear_trans = nn.Softplus()
    scatac = non_linear_trans(torch.tensor(scrna)).detach().numpy()
    print('scrna.shape: ', scrna.shape)
    print('')
    print('')
    print('scatac.shape:', scatac.shape)
    print('')
    print('')
    print('scrna:', scrna)
    print('')
    print('')
    print('scatac:', scatac)
    return scatac, scrna

In [3]:
X, y = data_generator()
print("Dimensions of input datasets are: ", "X= ", X.shape, " y= ", y.shape)

#Plot aligned domains, samples colored by cell types:
cellTypes_X=np.loadtxt("../data/SNAREseq_atac_types.txt")
cellTypes_y=np.loadtxt("../data/SNAREseq_rna_types.txt")
labels = ["H1", "GM", "BJ", "K562"]



scrna.shape:  (1047, 10)


scatac.shape: (1047, 10)


scrna: [[  6.6883364   -2.3696006    2.14278082 ...  -0.38985914   0.03180685
    0.16824296]
 [ -1.64704263   3.87629104  -2.57201982 ...  -0.56939318   0.77513025
    0.40657076]
 [ -3.22729615   4.55422489 -10.10911411 ...  -0.50277229   0.11965411
    0.70274427]
 ...
 [ -2.60360565   3.75489872  -6.05216412 ...   0.24619445  -1.72947077
   -3.24436   ]
 [ -0.94623495   4.54416359   2.29832621 ...   2.12374688  -1.05837995
    0.42656378]
 [ -2.49249336   4.43299686   1.91649138 ...   1.89995418  -2.62129623
    4.05023675]]


scatac: [[6.68958098e+00 8.94000856e-02 2.25372105e+00 ... 5.17097267e-01
  7.09177058e-01 7.80802708e-01]
 [1.76151488e-01 3.89680671e+00 7.36045937e-02 ... 4.48440760e-01
  1.15400753e+00 9.16954272e-01]
 [3.88981623e-02 4.56469256e+00 4.07060243e-05 ... 4.73031236e-01
  7.54762807e-01 1.10502057e+00]
 ...
 [7.13958207e-02 3.77803189e+00 2.35000142e-03 ... 8.23801811e-01
  1.63290152e-01 3.82524678e-02]


In [ ]:
C1_opt_model_1, C2_opt_model_1 = model(X,y, cellTypes_X,cellTypes_y, labels, 1e-1)

iter 0
It.  |Err         
-------------------
    0|3.953489e-04|
It.  |Err         
-------------------
    0|5.183839e-04|
It.  |Err         
-------------------
    0|3.006393e-04|
iter 1
It.  |Err         
-------------------
    0|3.954399e-04|
It.  |Err         
-------------------
    0|5.186549e-04|
It.  |Err         
-------------------
    0|3.006543e-04|
iter 2
It.  |Err         
-------------------
    0|3.954416e-04|
It.  |Err         
-------------------
    0|5.186515e-04|
It.  |Err         
-------------------
    0|3.006591e-04|
iter 3
It.  |Err         
-------------------
    0|3.954432e-04|
It.  |Err         
-------------------
    0|5.186480e-04|
It.  |Err         
-------------------
    0|3.006638e-04|
iter 4
It.  |Err         
-------------------
    0|3.954449e-04|
It.  |Err         
-------------------
    0|5.186446e-04|
It.  |Err         
-------------------
    0|3.006685e-04|
iter 5
It.  |Err         
-------------------
    0|3.954466e-04|
It.  |Err     

In [ ]:
p = ot.unif(C1_opt.shape[0])
q =  ot.unif(C2_opt.shape[0])
P = ot.gromov.entropic_gromov_wasserstein(C1_opt, C2_opt, p, q, loss_fun='square_loss', epsilon = 1e-2, log=False, verbose=True)

In [ ]:
# projection
#Projecting the first domain onto the second domain
y_new=normalize(y)
weights=np.sum(P,axis = 0)
X_new =np.matmul(P, normalize(y)) / weights[:, None]